In [307]:
import sys  
import time  
import math  
import uuid  
import numpy as np  
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objs as go
from numpy import errstate  
from sklearn.datasets import fetch_openml  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import OneHotEncoder  
from tqdm import tqdm

In [308]:
sys.setrecursionlimit(10000)  

In [309]:
class VarValue:
    def __init__(self, value, children=(), varname=''):
        self.varname = varname
        self.value = value
        self.derivative_to = {}
        self.children = children

    def __chain_rule(self, dSelfdx, child):
        if(child.varname[:5] != 'const'):
            for grandchild_varname in child.derivative_to:
                if(grandchild_varname != 'const'):
                    if(grandchild_varname in self.derivative_to):

                        self.derivative_to[grandchild_varname] += dSelfdx * child.derivative_to[grandchild_varname]
                    else:
                        self.derivative_to[grandchild_varname] = dSelfdx * child.derivative_to[grandchild_varname]
            if(len(self.derivative_to) == 0):
              raise ValueError(self.varname, child.varname)

    def relu(self):
        out = VarValue(max(0,self.value), children=(self,), varname='out_relu_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 0 if self.value <= 0 else 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def ln(self):
        out = VarValue(math.log(self.value), children=(self,), varname='out_ln_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 1/(self.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __mul__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value * other.value, children=(self, other), varname='out_mul_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx1 = other.value
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = self.value
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __add__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value + other.value, children=(self, other), varname='out_add_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx1 = 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = 1
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __pow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        try:
            with errstate(over='raise', invalid='raise'):
                result = float(self.value) ** float(other.value)
        except (OverflowError, FloatingPointError):
            if abs(float(self.value)) > 1:
                result = math.inf
            else:
                result = 0

        out = VarValue(result, children=(self, other), varname='out_pow_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx = other.value * self.value**(other.value-1)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return other + (-self)

    def __rpow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(other.value**self.value, children=(self, other), varname='out_rpow_'+str(uuid.uuid4()))
        if self.varname:
            dodx = other.value**self.value * math.log(other.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rtruediv__(self, other):
        return other * self**-1

    # Equality
    def __eq__(self, other):
        if isinstance(other, VarValue):
            return self.varname == other.varname
        return self.varname == other

    # Inequality
    def __ne__(self, other):
        return not self.__eq__(other)

    # Less than
    def __lt__(self, other):
        if isinstance(other, VarValue):
            return self.value < other.value
        return self.value < other

    # Less than or equal
    def __le__(self, other):
        if isinstance(other, VarValue):
            return self.value <= other.value
        return self.value <= other

    # Greater than
    def __gt__(self, other):
        if isinstance(other, VarValue):
            return self.value > other.value
        return self.value > other

    # Greater than or equal
    def __ge__(self, other):
        if isinstance(other, VarValue):
            return self.value >= other.value
        return self.value >= other
    
    def log(self):
        out = VarValue(math.log(self.value), children=(self,), varname='outlog'+str(uuid.uuid4()))
        if self.varname[:5] != 'const':
            dodx = 1 / self.value
            if len(self.children) == 0:
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def exp(self):
        out = VarValue(math.exp(self.value), children=(self,), varname='out_exp_'+str(uuid.uuid4()))
        if self.varname[:5] != 'const':
            dodx = math.exp(self.value)
            if len(self.children) == 0:
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out



In [ ]:
class Layer:
    def __init__(self,n_neurons=3, init='zero', activation='relu', weights=None, biases=None):
        # weights & biases ditambah di init buat memfasilitasi loading weight & bias
        self.n_neurons = n_neurons
        self.current_input_batch = None
        self.init = init    # zero/uniform/normal/xavier/he - harusnya gaperlu disini, ini di layer langsung harusnya
        self.weights = weights
        self.biases = biases
        self.activation = activation    # linear/relu/sigmoid/tanh/softmax/binary_step/leaky_relu/prelu/elu/swish(ini)/gelu(ini)
        self.learning_rate = None

        self.net = None
        self.out = None

    def __update_weights_dEdW(self, dEdW):
        clip_value = 1.0  # maksimal update yang diperbolehkan
        dEdW = np.clip(dEdW, -clip_value, clip_value)
        self.weights -= self.learning_rate * dEdW

    def __update_biases_dEdB(self, dEdB):
        clip_value = 1.0  # maksimal update yang diperbolehkan
        dEdB = np.clip(dEdB, -clip_value, clip_value)
        self.biases -= self.learning_rate * dEdB

    def __update_weights_err_term(self, err_term):
        for input in self.current_input_batch:
            for i in self.weights:
                for j in i:
                    self.weights += self.learning_rate*err_term[j]*self.input[i]

    def __update_biases_err_term(self, err_term):
        for i in self.weights:
            for j in i:
                self.weights += self.learning_rate*err_term[j]*1

    def forward(self, current_input_batch):
        self.current_input_batch = current_input_batch

        if(self.weights is None):
            
            if(self.init == 'zero'):
                self.weights = np.array([[VarValue(0,varname='w_'+str(uuid.uuid4())) for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(0,varname='b_'+str(uuid.uuid4())) for _ in range(self.n_neurons)])
                print("TESTT")
                print(self.weights)
                print(self.biases)

            # Ini semua diround soalnya hasil operasinya kegedean, kena warning wkwkwk. CMIIW yh harusnya berapa angka di belakang koma - @evelynnn04

            elif(self.init == 'uniform'):
                limit = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.uniform(-limit, limit), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.uniform(-limit, limit), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'normal':
                std = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.normal(0, std), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'xavier':
                std = np.sqrt(2 / (len(self.current_input_batch[0]) + self.n_neurons))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'he':
                std = np.sqrt(2 / len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])


        self.net = (np.dot(self.current_input_batch, self.weights)) + self.biases

        if(self.activation == 'linear'):
            self.out = self.net

        elif(self.activation == 'relu'):
            self.out = np.array([[net.relu() for net in row] for row in self.net])

        elif(self.activation == 'sigmoid'):
            for i in self.net:
                for j in i:
                    j.value = np.clip(j.value, -500, 500)
            self.out = 1 / (1 + (math.exp(1))**(-self.net))

        elif(self.activation == 'tanh'):
            self.out = (math.exp(1)**self.net - math.exp(1)**(-self.net))/(math.exp(1)**self.net + math.exp(1)**(-self.net))

        elif self.activation == 'softmax':
            exp_values = np.array([[net.exp() for net in row] for row in self.net])
            sums = np.array([[VarValue(sum(n.value for n in row), varname='sum_exp_'+str(uuid.uuid4()))] for row in exp_values])
            self.out = np.array([[exp_values[i][j] / sums[i][0] for j in range(len(exp_values[i]))] for i in range(len(exp_values))])


    def backward(self, next_weights=None, next_err_terms=None, target=None, sum_err_term_k_w=None, err=None, method='instant_deriv'):
        if method == 'instant_deriv':
            err_terms = np.zeros(len(self.out[0]))
            
            # Output layer case
            if target is not None:
                for j in range(len(self.out[0])):
                    err_terms[j] = self.out[0][j].value - target[0][j]  # dE/dNet = output - target untuk softmax & cce
                
            # Hidden layer case
            else:
                err_terms = np.zeros(len(self.out[0]))
                for j in range(len(self.out[0])):
                    sum_err = 0.0
                    for k in range(len(next_err_terms)):
                        sum_err += next_weights[j][k].value * next_err_terms[k]
                    # Jika aktivasi linear: derivatifnya = 1, jika relu maka derivatifnya = 1 atau 0
                    if self.activation == 'relu':
                        deriv = 1 if self.net[0][j].value > 0 else 0
                    else: # linear atau lainnya (TO-DO)
                        deriv = 1
                    err_terms[j] = sum_err * deriv

            # Hitung dE/dW dan dE/dB
            dEdW = np.zeros((len(self.weights), len(self.weights[0])))
            dEdB = np.zeros(len(self.biases))

            for i in range(len(self.weights)):
                for j in range(len(self.weights[i])):
                    dEdW[i][j] = self.current_input_batch[0][i].value * err_terms[j]

            for j in range(len(self.biases)):
                dEdB[j] = err_terms[j]

            # Update bobot dan bias
            self.__update_weights_dEdW(dEdW)
            self.__update_biases_dEdB(dEdB)

            return err_terms

        else:
            # Versi pake Error Term - Not Tested Yet
            if(err):
                # Output Layer
                err_term = np.array([err.derivative_to[net.varname] for net in self.net])
            else:
                # Hidden Layer
                err_term = sum_err_term_k_w
                pass
            sum_err_term_w_current = self.weights @ err_term.T
            self.__update_weights_err_term(err_term)
            self.__update_biases_err_term(err_term)
            return sum_err_term_w_current,

    def clean_derivative(self):
        for input in self.current_input_batch:
            for x in input:
                x.derivative_to.clear()
                x.children = ()

        for i in self.weights:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for b in self.biases:
            b.derivative_to ={}
            b.children = ()

        for i in self.net:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for i in self.out:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

In [311]:
class FFNN:
    def __init__(self, loss='mse', batch_size=1, learning_rate=0.01, epochs=20, verbose=0):
        self.loss = loss    # mse/bce/cce
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.verbose=verbose
        self.layers = None
        self.weights = []
        self.bias = []
        self.x = None
        self.y = None
        self.onehot_encoder = OneHotEncoder(categories='auto')

    def __loss(self, out, target):
        if(out.shape != target.shape):
            print("Output shape: ", out.shape)
            print("Target shape: ", target.shape)
            raise ValueError("Shape not match")
        if self.loss == 'mse':
            mse = (1/target.shape[1])*np.square(target - out)
            return np.sum(mse)
        elif self.loss == 'cce':
            epsilon = 1e-15
            loss = VarValue(0.0, varname='loss_const')
            for i in range(target.shape[0]):            # loop untuk batch
                for j in range(target.shape[1]):        # loop untuk neuron output
                    target_val = VarValue(target[i, j], varname=f'target_{i}_{j}')
                    out_clipped = VarValue(np.clip(out[i, j].value, epsilon, 1 - epsilon), varname=f'out_clipped_{i}_{j}')
                    loss = loss + (target_val * out_clipped.log())
            loss = loss * (-1 / self.batch_size)
            return loss

    def build_layers(self, *layers: Layer):
        self.layers = layers
        for layer in self.layers:
            layer.learning_rate = self.learning_rate

    def fit(self, x, y):
        self.x = x
        self.y = self.onehot_encoder.fit_transform(y.reshape(-1, 1))
        total_batch = (len(x)+self.batch_size-1)//self.batch_size
        start_global = time.time()

        if self.verbose == 1:
                epoch_iterator = tqdm(range(self.epochs), desc="Training Progress", unit="epoch")
        else:
            epoch_iterator = range(self.epochs)

        for epoch in epoch_iterator:
            print(f"Epoch {epoch+1}")
            start = time.time()
            # count_batch = 1

            if self.verbose == 1:
                batch_iterator = tqdm(range(total_batch), desc=f"Epoch {epoch + 1}", unit="batch", leave=False)
            else:
                batch_iterator = range(total_batch)
            
    
            for i in batch_iterator:
                x_batch = self.x[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.x[i*self.batch_size:]
                y_batch = self.y[i*self.batch_size:(i+1)*self.batch_size] if ((i+1) < total_batch) else self.y[i*self.batch_size:]

                batch_input = x_batch
                for layer in self.layers:
                    layer.forward(batch_input)
                    batch_input = layer.out
                out = batch_input

                # Backward (output layer dengan target)
                next_err_terms = self.layers[-1].backward(target=y_batch.toarray())

                # Backward hidden layers
                for idx in reversed(range(len(self.layers)-1)):
                    current_layer = self.layers[idx]
                    next_layer = self.layers[idx+1]
                    next_err_terms = current_layer.backward(next_weights=next_layer.weights, next_err_terms=next_err_terms)

                # Clean derivatives after batch selesai
                for layer in self.layers:
                    layer.clean_derivative()

        end_global = time.time()
        print("Total Duration: ", end_global-start_global)

    def predict(self, x_predict):
        batch_input = x_predict
        for layer in self.layers:
            layer.forward(batch_input)
            batch_input = layer.out
        out = batch_input
        return out

    def visualize(self):
        num_layers = len(self.layers)
        
        # Color palette
        layer_colors = {
            0: 'yellow',    # Input layer
            -1: 'salmon',   # Output layer
            'hidden': 'lightblue'  # Hidden layers
        }
        
        nodes_x = []
        nodes_y = []
        node_colors = []
        node_texts = []
        
        edges_x = []
        edges_y = []
        edge_texts = []
        
        for layer_idx, layer in enumerate(self.layers):
            if num_layers == 1:
                color = layer_colors[-1]
            elif layer_idx == 0:
                color = layer_colors[0]
            elif layer_idx == num_layers - 1:
                color = layer_colors[-1]
            else:
                color = layer_colors['hidden']
            
            n_neurons = layer.n_neurons
            y_positions = np.linspace(0, 1, n_neurons)
            x_pos = layer_idx / (num_layers - 1) if num_layers > 1 else 0.5
            
            for neuron_idx, y_pos in enumerate(y_positions):
                nodes_x.append(x_pos)
                nodes_y.append(y_pos)
                node_colors.append(color)
                node_texts.append(f"Layer {layer_idx}, Neuron {neuron_idx}<br>Activation: {layer.activation}")
            
            if layer_idx < num_layers - 1:
                next_layer = self.layers[layer_idx + 1]
                next_x = (layer_idx + 1) / (num_layers - 1) if num_layers > 1 else 0.5
                next_n_neurons = next_layer.n_neurons
                next_y_positions = np.linspace(0, 1, next_n_neurons)
                
                for curr_neuron_idx, curr_y in enumerate(y_positions):
                    for next_neuron_idx, next_y in enumerate(next_y_positions):
 
                        edges_x.extend([x_pos, next_x, None])
                        edges_y.extend([curr_y, next_y, None])
                        
                        weight_text = "Weight: Not initialized"
                        if layer.weights is not None:
                            try:
                                weight = layer.weights[curr_neuron_idx][next_neuron_idx]
                                weight_text = f"Weight: {weight}"
                            except:
                                weight_text = "Weight: Unavailable"
                        
                        edge_texts.append(weight_text)
        
        # Create nodes
        node_trace = go.Scatter(
            x=nodes_x, y=nodes_y,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=False,
                color=node_colors,
                size=15,
                line_width=2
            ),
            text=node_texts
        )
        
        # Create edges
        edge_trace = go.Scatter(
            x=edges_x, y=edges_y,
            mode='lines',
            line=dict(width=0.5, color='#888'),
            hoverinfo='text',
            text=edge_texts
        )
        
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title='Neural Network Architecture',
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0,l=0,r=0,t=40),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                        ))
        
        fig.update_layout(plot_bgcolor='rgba(255,255,255,255)')
        fig.show()

    
    def save(self, filename):
        model_data = {
            'loss': self.loss,
            'batch_size': self.batch_size,
            'learning_rate': self.learning_rate,
            'epochs': self.epochs,
            'layers': [
                {'n_neurons': layer.n_neurons, 'activation': layer.activation, 'weights': layer.weights.tolist(), 'biases': layer.biases.tolist()}
                for layer in self.layers
            ]
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to {filename}")

    def load(self, filename):
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        self.loss = model_data['loss']
        self.batch_size = model_data['batch_size']
        self.learning_rate = model_data['learning_rate']
        self.epochs = model_data['epochs']
        self.layers = [
            Layer(n_neurons=layer['n_neurons'], activation=layer['activation'], weights=np.array(layer['weights']), biases=np.array(layer['biases']))
            for layer in model_data['layers']
        ]
        print(f"Model loaded from {filename}")


In [312]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = y.astype(np.uint8)

In [313]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=65500,
    train_size=10,
    random_state=42,
    stratify=y
)

In [314]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [315]:
X_train = np.array([[VarValue(x, varname='x_'+str(uuid.uuid4())) for x in row] for row in X_train])

In [316]:
model_ffnn = FFNN(
    loss='cce',
    batch_size=1,
    learning_rate=0.1,
    epochs=2,
    verbose=1,
)

In [317]:
model_ffnn.build_layers(
    Layer(n_neurons=4, init='uniform', activation='relu'),
    Layer(n_neurons=3, init='uniform', activation='relu'),
    Layer(n_neurons=2, init='uniform', activation='relu'),
    Layer(n_neurons=10, init='uniform', activation='softmax')
)

In [318]:
model_ffnn.fit(X_train, y_train)

Training Progress:   0%|          | 0/2 [00:00<?, ?epoch/s]

Epoch 1


Training Progress:  50%|█████     | 1/2 [00:08<00:08,  8.41s/epoch]

Epoch 2


Training Progress: 100%|██████████| 2/2 [00:15<00:00,  7.65s/epoch]

Total Duration:  15.29583215713501


In [319]:
model_ffnn.visualize()

In [320]:
for layer in model_ffnn.layers:
    print("=====Layer======")
    for i in layer.weights:
        print("==Neuron==")
        for j in i:
            print(j.varname)

=====Layer======
==Neuron==
out_add_3595fd6e-b10f-4724-a8df-b5c484c3a2b5
out_add_be75a1af-60b3-4339-be2f-85361a5606a5
out_add_04f2d11d-561e-4f66-aa91-aa7c08d2dad5
out_add_fc962419-9d26-44b4-9e9d-a8819e6ee1f2
==Neuron==
out_add_41c890d6-cfb6-459d-8a4e-1fcdda7e8040
out_add_273e884f-7a57-48be-aebd-40c17bbc47ef
out_add_e6a9e1cd-0aa4-4ba1-b984-8eafca007acc
out_add_40ca47de-5982-4485-b021-40e34648d602
==Neuron==
out_add_a36e0eac-9e72-4fc8-bce1-cfa8d569fd02
out_add_3df76d5a-e9d7-4031-bbc0-1f99beea2be5
out_add_64652b06-e02b-4812-a7e5-14c8328518d0
out_add_4af23ba0-f6fd-49e6-b0d9-6dcc909ebf0b
==Neuron==
out_add_3553b5b1-2f3c-4db8-b21f-7bbbcf5692f0
out_add_b792da9a-e633-4f23-93cd-5efcfe9d6abe
out_add_0f11f461-80d7-47a1-9704-3c8452fe45be
out_add_9b9057f0-3339-4299-b728-3183576bc612
==Neuron==
out_add_c666f7a9-cdac-4b57-9070-de8d1b88f243
out_add_aa9f6ca3-5fbf-40d7-b605-f3909a94d2be
out_add_403abed2-c778-45d9-833b-d347cda8cd9c
out_add_c51f7b8f-eba2-4a92-92e6-c21b28ab76a4
==Neuron==
out_add_e7aed8dd-

In [321]:
for layer in model_ffnn.layers:
    for i in layer.weights:
        for j in i:
            print(j.value)

0.0232
-0.0076
-0.0046
-0.0052
-0.0023
-0.007
0.0064
0.0224
-0.0261
-0.0064
-0.032
0.0188
0.0181
0.0104
-0.0314
-0.0059
0.0349
0.0159
0.0086
0.0154
0.0331
-0.0228
0.0001
0.033
-0.0281
-0.0145
0.0065
-0.0242
0.0246
-0.021
0.0032
0.0178
0.0088
0.0235
-0.0209
-0.0302
-0.0151
-0.01
0.0105
-0.0074
-0.0065
-0.025
0.01
-0.0227
-0.0198
-0.0201
0.0029
-0.0176
-0.0355
-0.0092
0.0148
0.0178
-0.0317
0.0262
-0.0279
0.0346
0.0248
-0.022
-0.0308
0.0088
0.0319
-0.0124
-0.0104
-0.0113
0.0267
-0.0177
-0.0281
-0.013
-0.0189
-0.0245
-0.0285
0.004
-0.012
-0.0047
0.0244
-0.014
0.0135
0.0195
0.0086
0.0145
0.0166
-0.004
0.005
-0.0189
0.0137
-0.0279
0.0274
-0.0261
0.0228
-0.0248
-0.0321
0.0083
0.0338
-0.0327
-0.018
-0.035
0.0242
-0.0225
-0.0043
0.0227
-0.0164
0.0129
-0.0208
-0.0195
0.028
0.0287
0.001
0.0165
0.0215
-0.0103
0.0133
-0.0181
0.0296
-0.0053
-0.0068
-0.014
-0.0345
0.0203
0.0166
-0.0217
0.0111
0.0322
0.0086
-0.0343
-0.0296
0.0305
-0.0182
-0.0315
0.0287
0.0095
0.0331
-0.0075
0.0337
-0.0233
-0.0161
-0.0

In [322]:
y_pred = model_ffnn.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

6
6
6
6
6
6
6
6
6
6


In [323]:
model_ffnn.y.shape

(10, 10)

In [324]:
for h in y_test[:10].flatten():
    print(h)

9
6
5
9
6
9
8
8
1
2


In [325]:
model_ffnn.save(filename="try1.pth")

Model saved to try1.pth


In [326]:
new_model = FFNN()
new_model.load(filename="try1.pth")

Model loaded from try1.pth


In [327]:
y_pred = new_model.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

6
6
6
6
6
6
6
6
6
6
